# Entity Extraction Easy Mode w/ Pydantic

In [1]:
# Import utils like parsing between XML tags, invoking bedrock
%run pydantic-entity-extraction-demo-utils.ipynb

### Grab some sample text from which we want to extract entities

In [2]:
# TODO: use curl or something to grab text
document_text = open("wikipedia-text-example.txt", "r").read()

### Describe the entities you want to extract as Fields in a pydantic BaseModel

In [3]:
import pydantic
# from pydantic import BaseModel, Field, field_validator

print(pydantic.__version__)

2.6.3


In [4]:
import enum
from pydantic import BaseModel, Field, field_validator
from typing import Optional

class AgeEnum(enum.Enum):
    baby = "baby"
    child = "child"
    adult = "adult"

class Person(BaseModel):
    name: str = Field(
        ...,
        description=("The name of the person extracted. This field is required."),
        examples=["John Doe", "David Kaleko", "Rick Astley"],
    )
    birth_year: Optional[int] = Field(
        None,
        description=(
            "The year the extracted person was born, if known. This field is optional."
        ),
        examples=[1776, 1999],
    )
    age_class: Optional[AgeEnum] = Field(
        None,
        description=(
            "The age maturity of the person extracted. This field is optional. "
        ),
        examples=AgeEnum._member_names_,
    )
    job_title: Optional[str] = Field(
        None,
        description=("The job title of the person extracted. This field is optional."),
        examples=["Software Engineer", "Data Scientist", "Pirate King"],
    )


class URL(BaseModel):
    url: str = Field(
        ...,
        description=(
            "The full URL directly extracted from the provided text."
        ),
    )


class ExtractedEntities(BaseModel):
    people: list[Person] = Field(
        default=[],
        description=(
            "A list of people extracted from the text. The people are ordered by the "
            "order in which they appear in the text."
        ),
    )

    urls: list[URL] = Field(
        default=[],
        description=("A list of all website URLs extracted from the text."),
    )

    @field_validator("urls", mode="after")
    def validate_url(cls, values: list[URL]) -> list[URL]:
        """Drop any urls which don't contain "www." """
        if not values:
            return []
        return [u for u in values if "www." in u.url]

### Build a prompt to extract entities

In [5]:
PROMPT_PREFIX = (
    "Your goal is to extract structured information from a provided block of source_text that matches "
    "the form described below."
)
FORMAT_INSTRUCTIONS = (
    "Please output the extracted information in JSON format. Do not output anything except "
    "for the extracted information. Do not add any clarifying information. Do not add any "
    "fields that are not in the schema. "
    "All output must be in JSON format and follow the schema specified above. "
    "Wrap the JSON in <json> tags. "
)


def build_full_prompt(source_text: str) -> str:
    prompt = PROMPT_PREFIX
    prompt += f"""
    <schema>
        {ExtractedEntities.model_json_schema()}
    </schema>
    """
    prompt += f"<source_text>{source_text}</source_text>"
    prompt += FORMAT_INSTRUCTIONS + "\n\n"

    return prompt

### Send the prompt to an LLM and see if it works

In [6]:
full_prompt = build_full_prompt(source_text=document_text)
print(full_prompt)


Your goal is to extract structured information from a provided block of source_text that matches the form described below.
    <schema>
        {'$defs': {'AgeEnum': {'enum': ['baby', 'child', 'adult'], 'title': 'AgeEnum', 'type': 'string'}, 'Person': {'properties': {'name': {'description': 'The name of the person extracted. This field is required.', 'examples': ['John Doe', 'David Kaleko', 'Rick Astley'], 'title': 'Name', 'type': 'string'}, 'birth_year': {'anyOf': [{'type': 'integer'}, {'type': 'null'}], 'default': None, 'description': 'The year the extracted person was born, if known. This field is optional.', 'examples': [1776, 1999], 'title': 'Birth Year'}, 'age_class': {'anyOf': [{'$ref': '#/$defs/AgeEnum'}, {'type': 'null'}], 'default': None, 'description': 'The age maturity of the person extracted. This field is optional. ', 'examples': ['baby', 'child', 'adult']}, 'job_title': {'anyOf': [{'type': 'string'}, {'type': 'null'}], 'default': None, 'description': 'The job title of th

In [7]:
# model_id = "anthropic.claude-v2"
# model_id = "anthropic.claude-v2:1"
model_id = "anthropic.claude-3-sonnet-20240229-v1:0"
# model_id = "anthropic.claude-3-haiku-20240307-v1:0"
# model_id = "anthropic.claude-instant-v1"            # This one struggled

llm_response = invoke_claude_3_with_text(full_prompt, model_id=model_id, temperature=0)

In [8]:
print(llm_response)

<json>
{
  "people": [
    {
      "name": "Andrew R. Jassy",
      "birth_year": 1968,
      "age_class": "adult",
      "job_title": "president and chief executive officer (CEO) of Amazon"
    },
    {
      "name": "Jeff Bezos",
      "age_class": null,
      "job_title": null
    },
    {
      "name": "Margery Jassy",
      "age_class": null,
      "job_title": null
    },
    {
      "name": "Everett L. Jassy",
      "age_class": null,
      "job_title": "senior partner in the corporate law firm Dewey Ballantine in New York City"
    },
    {
      "name": "Elana Rochelle Caplan",
      "age_class": null,
      "job_title": "fashion designer for Eddie Bauer"
    },
    {
      "name": "James Brandt",
      "age_class": null,
      "job_title": "New York reformed Rabbi"
    },
    {
      "name": "Adam Selipsky",
      "age_class": null,
      "job_title": "CEO of AWS"
    }
  ],
  "urls": [
    {
      "url": "www.thecrimson.com"
    },
    {
      "url": "ww.thisisawebsite.com"


In [9]:
extracted_meta = ExtractedEntities.model_validate_json(
    extract_json_content(llm_response)
)

In [10]:
from devtools import pprint

pprint(extracted_meta)

ExtractedEntities(
    people=[
        Person(
            name='Andrew R. Jassy',
            birth_year=1968,
            age_class=<AgeEnum.adult: 'adult'>,
            job_title='president and chief executive officer (CEO) of Amazon',
        ),
        Person(
            name='Jeff Bezos',
            birth_year=None,
            age_class=None,
            job_title=None,
        ),
        Person(
            name='Margery Jassy',
            birth_year=None,
            age_class=None,
            job_title=None,
        ),
        Person(
            name='Everett L. Jassy',
            birth_year=None,
            age_class=None,
            job_title='senior partner in the corporate law firm Dewey Ballantine in New York City',
        ),
        Person(
            name='Elana Rochelle Caplan',
            birth_year=None,
            age_class=None,
            job_title='fashion designer for Eddie Bauer',
        ),
        Person(
            name='James Brandt',
      

In [11]:
# Demonstrating field validator
print(ExtractedEntities(urls=[URL(url="ww.amazon.com")]))
print(ExtractedEntities(urls=[URL(url="www.amazon.com")]))

people=[] urls=[]
people=[] urls=[URL(url='www.amazon.com')]
